In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps
sns.set()

plt.rc('font', size=30)
plt.rc('axes', titlesize=30)
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('legend', fontsize=30)
plt.rc('figure', titlesize=30)

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.covariance import MinCovDet
import cvxpy as cvx
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [21]:
%load_ext autoreload
%autoreload 2

In [40]:
returns = pd.read_csv('work/data/returns.csv')
returns = returns.fillna(0)
returns['date'] = returns['date'].astype(np.datetime64)
returns = returns.set_index('date')

train_size = 1500

returns_train = returns[:train_size]
returns_test = returns[train_size:]

In [72]:
np.seterr(all='raise', under='ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [42]:
A = np.arange(9).reshape(3, 3)
A.sum(1)

array([ 3, 12, 21])

In [62]:
np.exp(-100)
np.exp(-700)
np.exp(700)

1.0142320547350045e+304

In [80]:
float(1e-6)

1e-06

In [100]:
from mppca import MPPCA

mean = returns_train[returns_train.columns].mean()
top_returns = returns_train[mean.sort_values(ascending=False)[:25].index]

mppca = MPPCA(5, 4)
mppca.fit(top_returns.to_numpy())
mppca_top_returns = mppca.transform(top_returns.to_numpy())

np.mean(mppca_top_returns - top_returns.to_numpy()**2)

0.0011017057647423607

In [109]:
np.seterr(all='ignore')

{'divide': 'raise', 'over': 'raise', 'under': 'ignore', 'invalid': 'raise'}

In [110]:
for i in range(10, 100):
    mean = returns_train[returns_train.columns].mean()
    top_returns = returns_train[mean.sort_values(ascending=False)[:i].index]

    mppca = MPPCA(5, 4)
    mppca.fit(top_returns.to_numpy())
    mppca_top_returns = mppca.transform(top_returns.to_numpy())

    print(np.mean(mppca_top_returns - top_returns.to_numpy()**2), i)

0.0012021348000474902 10
0.0014106482161611526 11
0.0011359676659147685 12
0.001112855778122669 13
nan 14
0.0009792660908028504 15
0.0008763142139376427 16
0.001139722886945492 17
nan 18
0.0011437314719804036 19
nan 20
nan 21
nan 22
-0.002684362078342737 23
0.0009348850012725066 24
0.0010872921287348244 25
0.0009479351885853869 26
nan 27
0.0009371859280126123 28
-0.00010271099916815382 29
0.0009335976028454429 30
0.0009299135483910277 31
0.00010475846290136996 32
nan 33
nan 34
-0.0011361791589047817 35
0.001065966682278191 36
nan 37
nan 38
nan 39
-0.0019896238606693876 40
0.00104988222133456 41
0.0010560120312162937 42
6.19541822590104e-05 43
nan 44
0.00035866586947588895 45
nan 46
-0.0019772168223938114 47
0.00098464161161626 48
-0.001482005521169956 49
0.0009598066770326098 50
0.000961381269539077 51
0.00099396723536862 52
0.0010279065912558024 53
-0.0007021136702421048 54
-0.001076261915586871 55
0.0010305082966163889 56
0.0010382522056501349 57
0.0010937922313624723 58
0.0011928678

KeyboardInterrupt: 